<a href="https://colab.research.google.com/github/masonnystrom/DS-Unit-1-Sprint-1-Data-Wrangling-and-Storytelling/blob/master/module2-make-features/LS_DS_112_Make_Features_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200> 

# Assignment:

- Replicate the lesson code.

 - This means that if you haven't followed along already, type out the things that we did in class. Forcing your fingers to hit each key will help you internalize the syntax of what we're doing. Make sure you understand each line of code that you're writing, google things that you don't fully understand.
 - [Lambda Learning Method for DS - By Ryan Herr](https://docs.google.com/document/d/1ubOw9B3Hfip27hF2ZFnW3a3z9xAgrUDRReOEo-FHCVs/edit?usp=sharing)
- Convert the `term` column from string to integer.
- Make a column named `loan_status_is_great`. It should contain the integer 1 if `loan_status` is "Current" or "Fully Paid." Else it should contain the integer 0.
- Make `last_pymnt_d_month` and `last_pymnt_d_year` columns.

In [1]:
!wget https://resources.lendingclub.com/LoanStats_2018Q4.csv.zip

--2020-02-11 20:14:07--  https://resources.lendingclub.com/LoanStats_2018Q4.csv.zip
Resolving resources.lendingclub.com (resources.lendingclub.com)... 64.48.1.20
Connecting to resources.lendingclub.com (resources.lendingclub.com)|64.48.1.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘LoanStats_2018Q4.csv.zip’

LoanStats_2018Q4.cs     [                <=> ]  21.74M   890KB/s    in 25s     

2020-02-11 20:14:32 (891 KB/s) - ‘LoanStats_2018Q4.csv.zip’ saved [22793561]



In [2]:
!unzip LoanStats_2018Q4.csv.zip

Archive:  LoanStats_2018Q4.csv.zip
  inflating: LoanStats_2018Q4.csv    


In [5]:
import pandas as pd

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
df = pd.read_csv('LoanStats_2018Q4.csv', skiprows=1, skipfooter=2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [9]:
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,35000,35000,35000.0,36 months,14.47%,1204.23,C,C2,Staff Physician,8 years,MORTGAGE,360000.0,Verified,Dec-2018,Fully Paid,n,NaN,NaN,credit_card,Credit card refinancing,336xx,FL,19.90,0,Apr-1995,1,NaN,NaN,24,0,57259,43.2%,51,w,0.00,0.00,38187.046837,38187.05,...,30.8,0,0,1222051,169286,124600,258401,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,5000,5000,5000.0,36 months,22.35%,191.86,D,D5,Director of Sales,10+ years,MORTGAGE,72000.0,Source Verified,Dec-2018,Fully Paid,n,NaN,NaN,debt_consolidation,Debt consolidation,333xx,FL,20.12,0,Mar-2010,0,NaN,NaN,13,0,11720,47.1%,26,f,0.00,0.00,5615.977674,5615.98,...,50.0,0,0,218686,34418,18200,37786,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,10000,10000,10000.0,60 months,23.40%,284.21,E,E1,NaN,< 1 year,RENT,55000.0,Source Verified,Dec-2018,Current,n,NaN,NaN,debt_consolidation,Debt consolidation,902xx,CA,13.51,0,Apr-2007,0,44.0,88.0,9,1,11859,53.9%,11,w,8694.39,8694.39,3675.230000,3675.23,...,100.0,1,0,34386,21235,10500,12386,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,17100,17100,17100.0,36 months,18.94%,626.30,D,D2,Receptionist,10+ years,RENT,38000.0,Verified,Dec-2018,Current,n,NaN,NaN,debt_consolidation,Debt consolidation,150xx,PA,38.09,0,Mar-1998,1,47.0,NaN,14,0,15323,53%,21,w,12001.54,12001.54,8114.910000,8114.91,...,75.0,0,0,70954,43351,16600,41784,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,4000,4000,4000.0,36 months,10.72%,130.43,B,B2,Extrusion assistant,10+ years,MORTGAGE,56000.0,Verified,Dec-2018,Current,n,NaN,NaN,credit_card,Credit card refinancing,301xx,GA,31.03,0,Sep-2006,0,NaN,NaN,7,0,4518,28.6%,11,w,2700.74,2700.74,1682.500000,1682.50,...,0.0,0,0,221310,71375,12300,77865,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
pd.set_option('display.max_rows', 150)

In [20]:
print(df.shape)

(128412, 144)


In [0]:
df.isnull().sum()

In [0]:
loan_df = df.drop(['id', 'member_id', 'url', 'desc'], axis=1)

In [24]:
loan_df.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,35000,35000,35000.0,36 months,14.47%,1204.23,C,C2,Staff Physician,8 years,MORTGAGE,360000.0,Verified,Dec-2018,Fully Paid,n,credit_card,Credit card refinancing,336xx,FL,19.90,0,Apr-1995,1,NaN,NaN,24,0,57259,43.2%,51,w,0.00,0.00,38187.046837,38187.05,35000.00,3187.05,0.0,0.0,...,30.8,0,0,1222051,169286,124600,258401,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
1,5000,5000,5000.0,36 months,22.35%,191.86,D,D5,Director of Sales,10+ years,MORTGAGE,72000.0,Source Verified,Dec-2018,Fully Paid,n,debt_consolidation,Debt consolidation,333xx,FL,20.12,0,Mar-2010,0,NaN,NaN,13,0,11720,47.1%,26,f,0.00,0.00,5615.977674,5615.98,5000.00,615.98,0.0,0.0,...,50.0,0,0,218686,34418,18200,37786,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
2,10000,10000,10000.0,60 months,23.40%,284.21,E,E1,NaN,< 1 year,RENT,55000.0,Source Verified,Dec-2018,Current,n,debt_consolidation,Debt consolidation,902xx,CA,13.51,0,Apr-2007,0,44.0,88.0,9,1,11859,53.9%,11,w,8694.39,8694.39,3675.230000,3675.23,1305.61,2369.62,0.0,0.0,...,100.0,1,0,34386,21235,10500,12386,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
3,17100,17100,17100.0,36 months,18.94%,626.30,D,D2,Receptionist,10+ years,RENT,38000.0,Verified,Dec-2018,Current,n,debt_consolidation,Debt consolidation,150xx,PA,38.09,0,Mar-1998,1,47.0,NaN,14,0,15323,53%,21,w,12001.54,12001.54,8114.910000,8114.91,5098.46,3016.45,0.0,0.0,...,75.0,0,0,70954,43351,16600,41784,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
4,4000,4000,4000.0,36 months,10.72%,130.43,B,B2,Extrusion assistant,10+ years,MORTGAGE,56000.0,Verified,Dec-2018,Current,n,credit_card,Credit card refinancing,301xx,GA,31.03,0,Sep-2006,0,NaN,NaN,7,0,4518,28.6%,11,w,2700.74,2700.74,1682.500000,1682.50,1299.26,383.24,0.0,0.0,...,0.0,0,0,221310,71375,12300,77865,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
df['int_rate'].describe()

count      128412
unique         46
top        13.56%
freq         6972
Name: int_rate, dtype: object

In [0]:
# For loop to fix int rate
new_int_rate = []

for cell in df['int_rate']:
  new_int_rate.append(float(cell.strip().strip('%')))

In [0]:
new_int_rate

In [0]:
# function to fix string to int

def int_rate_to_float(cell_contents):
  return float(cell_contents.strip().strip('%'))

In [0]:
loan_df['int_rate_float'] = df['int_rate'].apply(int_rate_to_float)

In [35]:
loan_df.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,...,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,int_rate_float
0,35000,35000,35000.0,36 months,14.47%,1204.23,C,C2,Staff Physician,8 years,MORTGAGE,360000.0,Verified,Dec-2018,Fully Paid,n,credit_card,Credit card refinancing,336xx,FL,19.90,0,Apr-1995,1,NaN,NaN,24,0,57259,43.2%,51,w,0.00,0.00,38187.046837,38187.05,35000.00,3187.05,0.0,0.0,...,0,0,1222051,169286,124600,258401,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,14.47
1,5000,5000,5000.0,36 months,22.35%,191.86,D,D5,Director of Sales,10+ years,MORTGAGE,72000.0,Source Verified,Dec-2018,Fully Paid,n,debt_consolidation,Debt consolidation,333xx,FL,20.12,0,Mar-2010,0,NaN,NaN,13,0,11720,47.1%,26,f,0.00,0.00,5615.977674,5615.98,5000.00,615.98,0.0,0.0,...,0,0,218686,34418,18200,37786,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,22.35
2,10000,10000,10000.0,60 months,23.40%,284.21,E,E1,NaN,< 1 year,RENT,55000.0,Source Verified,Dec-2018,Current,n,debt_consolidation,Debt consolidation,902xx,CA,13.51,0,Apr-2007,0,44.0,88.0,9,1,11859,53.9%,11,w,8694.39,8694.39,3675.230000,3675.23,1305.61,2369.62,0.0,0.0,...,1,0,34386,21235,10500,12386,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,23.40
3,17100,17100,17100.0,36 months,18.94%,626.30,D,D2,Receptionist,10+ years,RENT,38000.0,Verified,Dec-2018,Current,n,debt_consolidation,Debt consolidation,150xx,PA,38.09,0,Mar-1998,1,47.0,NaN,14,0,15323,53%,21,w,12001.54,12001.54,8114.910000,8114.91,5098.46,3016.45,0.0,0.0,...,0,0,70954,43351,16600,41784,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,18.94
4,4000,4000,4000.0,36 months,10.72%,130.43,B,B2,Extrusion assistant,10+ years,MORTGAGE,56000.0,Verified,Dec-2018,Current,n,credit_card,Credit card refinancing,301xx,GA,31.03,0,Sep-2006,0,NaN,NaN,7,0,4518,28.6%,11,w,2700.74,2700.74,1682.500000,1682.50,1299.26,383.24,0.0,0.0,...,0,0,221310,71375,12300,77865,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,10.72


In [36]:
loan_df['emp_title'].value_counts(dropna=False).head(20)

NaN                   20947
Teacher                2090
Manager                1773
Registered Nurse        952
Driver                  924
RN                      726
Supervisor              697
Sales                   580
Project Manager         526
General Manager         523
Office Manager          521
Owner                   420
Director                402
Truck Driver            387
Operations Manager      387
Nurse                   326
Engineer                325
Sales Manager           304
manager                 301
Supervisor              270
Name: emp_title, dtype: int64

In [37]:
len(loan_df['emp_title'].value_counts())

43892

In [39]:
loan_df['emp_title'].nunique()

43892

In [40]:
loan_df['emp_title'].isnull().sum()

20947

In [0]:
# clean the emp_title data
import numpy as np

In [0]:
# funciton for cleaning titles

def clean_title(title):
  if isinstance(title, str):
    return title.strip().title()
  else:
    return "Unknown"

In [43]:
loan_df['emp_title'] = loan_df['emp_title'].apply(clean_title)

loan_df.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,...,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,int_rate_float
0,35000,35000,35000.0,36 months,14.47%,1204.23,C,C2,Staff Physician,8 years,MORTGAGE,360000.0,Verified,Dec-2018,Fully Paid,n,credit_card,Credit card refinancing,336xx,FL,19.90,0,Apr-1995,1,NaN,NaN,24,0,57259,43.2%,51,w,0.00,0.00,38187.046837,38187.05,35000.00,3187.05,0.0,0.0,...,0,0,1222051,169286,124600,258401,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,14.47
1,5000,5000,5000.0,36 months,22.35%,191.86,D,D5,Director Of Sales,10+ years,MORTGAGE,72000.0,Source Verified,Dec-2018,Fully Paid,n,debt_consolidation,Debt consolidation,333xx,FL,20.12,0,Mar-2010,0,NaN,NaN,13,0,11720,47.1%,26,f,0.00,0.00,5615.977674,5615.98,5000.00,615.98,0.0,0.0,...,0,0,218686,34418,18200,37786,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,22.35
2,10000,10000,10000.0,60 months,23.40%,284.21,E,E1,Unknown,< 1 year,RENT,55000.0,Source Verified,Dec-2018,Current,n,debt_consolidation,Debt consolidation,902xx,CA,13.51,0,Apr-2007,0,44.0,88.0,9,1,11859,53.9%,11,w,8694.39,8694.39,3675.230000,3675.23,1305.61,2369.62,0.0,0.0,...,1,0,34386,21235,10500,12386,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,23.40
3,17100,17100,17100.0,36 months,18.94%,626.30,D,D2,Receptionist,10+ years,RENT,38000.0,Verified,Dec-2018,Current,n,debt_consolidation,Debt consolidation,150xx,PA,38.09,0,Mar-1998,1,47.0,NaN,14,0,15323,53%,21,w,12001.54,12001.54,8114.910000,8114.91,5098.46,3016.45,0.0,0.0,...,0,0,70954,43351,16600,41784,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,18.94
4,4000,4000,4000.0,36 months,10.72%,130.43,B,B2,Extrusion Assistant,10+ years,MORTGAGE,56000.0,Verified,Dec-2018,Current,n,credit_card,Credit card refinancing,301xx,GA,31.03,0,Sep-2006,0,NaN,NaN,7,0,4518,28.6%,11,w,2700.74,2700.74,1682.500000,1682.50,1299.26,383.24,0.0,0.0,...,0,0,221310,71375,12300,77865,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,10.72


In [44]:
# checking the emp data
loan_df['emp_title'].value_counts(dropna=False).head(20)

Unknown               20947
Teacher                2557
Manager                2395
Registered Nurse       1418
Driver                 1258
Supervisor             1160
Truck Driver            920
Rn                      834
Office Manager          805
Sales                   803
General Manager         791
Project Manager         720
Owner                   625
Director                523
Operations Manager      518
Sales Manager           500
Police Officer          440
Nurse                   425
Technician              420
Engineer                412
Name: emp_title, dtype: int64

In [45]:
len(loan_df['emp_title'].value_counts())

34902

In [46]:
loan_df['emp_title'].isnull().sum()

0

In [49]:
# changing dates from lecture
[col for col in loan_df if col.endswith('_d')]

['issue_d', 'last_pymnt_d', 'next_pymnt_d', 'last_credit_pull_d']

In [51]:
loan_df['issue_d'].head()

0    Dec-2018
1    Dec-2018
2    Dec-2018
3    Dec-2018
4    Dec-2018
Name: issue_d, dtype: object

In [0]:
loan_df['issue_d'] = pd.to_datetime(loan_df['issue_d'], infer_datetime_format=True)

In [0]:
loan_df.dtypes

In [54]:
loan_df['issue_d'].head()

0   2018-12-01
1   2018-12-01
2   2018-12-01
3   2018-12-01
4   2018-12-01
Name: issue_d, dtype: datetime64[ns]

In [55]:
loan_df['issue_d'].dt.year

0         2018
1         2018
2         2018
3         2018
4         2018
          ... 
128407    2018
128408    2018
128409    2018
128410    2018
128411    2018
Name: issue_d, Length: 128412, dtype: int64

In [56]:
loan_df['issue_d'].dt.month

0         12
1         12
2         12
3         12
4         12
          ..
128407    10
128408    10
128409    10
128410    10
128411    10
Name: issue_d, Length: 128412, dtype: int64

In [0]:
loan_df['issue_year'] = loan_df['issue_d'].dt.year 
loan_df['issue_month'] = loan_df['issue_d'].dt.month

In [58]:
loan_df['earliest_cr_line'].head()

0    Apr-1995
1    Mar-2010
2    Apr-2007
3    Mar-1998
4    Sep-2006
Name: earliest_cr_line, dtype: object

In [0]:
loan_df["earliest_cr_line"] = pd.to_datetime(loan_df['earliest_cr_line'], infer_datetime_format=True)

In [0]:
loan_df['Credit History'] = (loan_df['issue_d'] - loan_df['earliest_cr_line']).dt.days

In [63]:
loan_df['Credit History'].describe() / 365


count    351.813699
mean      16.054497
std        7.908317
min        3.084932
25%       11.093151
50%       14.427397
75%       19.846575
max       68.961644
Name: Credit History, dtype: float64

In [0]:
# Replication complete

In [0]:
#Convert the term column from string to integer.
test = loan_df['term'][1]

In [80]:
int(test.strip().strip('months'))

36

In [0]:
def string_to_int(cell_input):
  return int(cell_input.strip().strip('months'))

In [0]:
loan_df['term'] = loan_df['term'].apply(string_to_int)

In [83]:
loan_df.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,...,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,int_rate_float,issue_year,issue_month,Credit History
0,35000,35000,35000.0,36,14.47%,1204.23,C,C2,Staff Physician,8 years,MORTGAGE,360000.0,Verified,2018-12-01,Fully Paid,n,credit_card,Credit card refinancing,336xx,FL,19.90,0,1995-04-01,1,NaN,NaN,24,0,57259,43.2%,51,w,0.00,0.00,38187.046837,38187.05,35000.00,3187.05,0.0,0.0,...,169286,124600,258401,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,14.47,2018,12,8645
1,5000,5000,5000.0,36,22.35%,191.86,D,D5,Director Of Sales,10+ years,MORTGAGE,72000.0,Source Verified,2018-12-01,Fully Paid,n,debt_consolidation,Debt consolidation,333xx,FL,20.12,0,2010-03-01,0,NaN,NaN,13,0,11720,47.1%,26,f,0.00,0.00,5615.977674,5615.98,5000.00,615.98,0.0,0.0,...,34418,18200,37786,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,22.35,2018,12,3197
2,10000,10000,10000.0,60,23.40%,284.21,E,E1,Unknown,< 1 year,RENT,55000.0,Source Verified,2018-12-01,Current,n,debt_consolidation,Debt consolidation,902xx,CA,13.51,0,2007-04-01,0,44.0,88.0,9,1,11859,53.9%,11,w,8694.39,8694.39,3675.230000,3675.23,1305.61,2369.62,0.0,0.0,...,21235,10500,12386,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,23.40,2018,12,4262
3,17100,17100,17100.0,36,18.94%,626.30,D,D2,Receptionist,10+ years,RENT,38000.0,Verified,2018-12-01,Current,n,debt_consolidation,Debt consolidation,150xx,PA,38.09,0,1998-03-01,1,47.0,NaN,14,0,15323,53%,21,w,12001.54,12001.54,8114.910000,8114.91,5098.46,3016.45,0.0,0.0,...,43351,16600,41784,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,18.94,2018,12,7580
4,4000,4000,4000.0,36,10.72%,130.43,B,B2,Extrusion Assistant,10+ years,MORTGAGE,56000.0,Verified,2018-12-01,Current,n,credit_card,Credit card refinancing,301xx,GA,31.03,0,2006-09-01,0,NaN,NaN,7,0,4518,28.6%,11,w,2700.74,2700.74,1682.500000,1682.50,1299.26,383.24,0.0,0.0,...,71375,12300,77865,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,10.72,2018,12,4474


In [86]:
# Make a column named loan_status_is_great. It should contain the integer 1 if loan_status is "Current" or "Fully Paid." Else it should contain the integer 0.
loan_df['loan_status'].isnull().sum()

0

In [0]:
def loanstatus(row):
  if row == 'Fully Paid':
    return row.replace("Fully Paid",'1')
  elif row == "Current":
    return row.replace("Current", '1')
  else:
    return '0'

In [0]:
loan_df['loan_status_is_great'] = loan_df['loan_status'].apply(loanstatus)

In [271]:
loan_df.head(10)

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,...,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,int_rate_float,issue_year,issue_month,Credit History,loan_status_is_great,last_pymnt_d_month,last_pymnt_d_year,revol_util_int,revol_util_float,top20,revol_util_loans,loan_status_is_great1
0,35000,35000,35000.0,36,14.47%,1204.23,C,C2,Staff Physician,8 years,MORTGAGE,360000.0,Verified,2018-12-01,Fully Paid,n,credit_card,Credit card refinancing,336xx,FL,19.90,0,1995-04-01,1,NaN,NaN,24,0,57259,43.2%,51,w,0.00,0.00,38187.046837,38187.05,35000.00,3187.05,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,14.47,2018,12,8645,1,8.0,2019.0,43.2,43.2,Unknown,43.2,1
1,5000,5000,5000.0,36,22.35%,191.86,D,D5,Director Of Sales,10+ years,MORTGAGE,72000.0,Source Verified,2018-12-01,Fully Paid,n,debt_consolidation,Debt consolidation,333xx,FL,20.12,0,2010-03-01,0,NaN,NaN,13,0,11720,47.1%,26,f,0.00,0.00,5615.977674,5615.98,5000.00,615.98,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,22.35,2018,12,3197,1,7.0,2019.0,47.1,47.1,Unknown,47.1,1
2,10000,10000,10000.0,60,23.40%,284.21,E,E1,Unknown,< 1 year,RENT,55000.0,Source Verified,2018-12-01,Current,n,debt_consolidation,Debt consolidation,902xx,CA,13.51,0,2007-04-01,0,44.0,88.0,9,1,11859,53.9%,11,w,8694.39,8694.39,3675.230000,3675.23,1305.61,2369.62,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,23.40,2018,12,4262,1,1.0,2020.0,53.9,53.9,Unknown,53.9,1
3,17100,17100,17100.0,36,18.94%,626.30,D,D2,Receptionist,10+ years,RENT,38000.0,Verified,2018-12-01,Current,n,debt_consolidation,Debt consolidation,150xx,PA,38.09,0,1998-03-01,1,47.0,NaN,14,0,15323,53%,21,w,12001.54,12001.54,8114.910000,8114.91,5098.46,3016.45,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,18.94,2018,12,7580,1,1.0,2020.0,53,53,Unknown,53.0,1
4,4000,4000,4000.0,36,10.72%,130.43,B,B2,Extrusion Assistant,10+ years,MORTGAGE,56000.0,Verified,2018-12-01,Current,n,credit_card,Credit card refinancing,301xx,GA,31.03,0,2006-09-01,0,NaN,NaN,7,0,4518,28.6%,11,w,2700.74,2700.74,1682.500000,1682.50,1299.26,383.24,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,10.72,2018,12,4474,1,1.0,2020.0,28.6,28.6,Unknown,28.6,1
5,10475,10475,10475.0,36,11.31%,344.48,B,B3,Teacher,6 years,RENT,66150.0,Source Verified,2018-12-01,Current,n,debt_consolidation,Debt consolidation,786xx,TX,7.40,0,1991-06-01,0,28.0,NaN,10,0,10499,40.2%,17,w,7093.15,7093.15,4468.370000,4468.37,3381.85,1086.52,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,11.31,2018,12,10045,1,1.0,2020.0,40.2,40.2,Unknown,40.2,1
6,7500,7500,7500.0,36,11.31%,246.65,B,B3,Supervisor,7 years,MORTGAGE,40500.

In [0]:
# Make last_pymnt_d_month and last_pymnt_d_year columns.

In [91]:
loan_df['last_pymnt_d']

0         Aug-2019
1         Jul-2019
2         Jan-2020
3         Jan-2020
4         Jan-2020
            ...   
128407    Dec-2018
128408    Jan-2020
128409    Jan-2020
128410    Jan-2020
128411    Jan-2020
Name: last_pymnt_d, Length: 128412, dtype: object

In [0]:
loan_df['last_pymnt_d'] = pd.to_datetime(loan_df['last_pymnt_d'],infer_datetime_format=True)

In [93]:
loan_df['last_pymnt_d']

0        2019-08-01
1        2019-07-01
2        2020-01-01
3        2020-01-01
4        2020-01-01
            ...    
128407   2018-12-01
128408   2020-01-01
128409   2020-01-01
128410   2020-01-01
128411   2020-01-01
Name: last_pymnt_d, Length: 128412, dtype: datetime64[ns]

In [97]:
loan_df['last_pymnt_d_month'] = (loan_df['last_pymnt_d'].dt.month)
loan_df['last_pymnt_d_month'].head()

0    8.0
1    7.0
2    1.0
3    1.0
4    1.0
Name: last_pymnt_d_month, dtype: float64

In [98]:
loan_df['last_pymnt_d_year'] = loan_df['last_pymnt_d'].dt.year
loan_df['last_pymnt_d_year'].head()

0    2019.0
1    2019.0
2    2020.0
3    2020.0
4    2020.0
Name: last_pymnt_d_year, dtype: float64

In [116]:
# Stretch goals for Lending Club Dataset
loan_df.head(10)

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,...,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,int_rate_float,issue_year,issue_month,Credit History,loan_status_is_great,last_pymnt_d_month,last_pymnt_d_year
0,35000,35000,35000.0,36,14.47%,1204.23,C,C2,Staff Physician,8 years,MORTGAGE,360000.0,Verified,2018-12-01,Fully Paid,n,credit_card,Credit card refinancing,336xx,FL,19.90,0,1995-04-01,1,NaN,NaN,24,0,57259,43.2%,51,w,0.00,0.00,38187.046837,38187.05,35000.00,3187.05,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,14.47,2018,12,8645,0,8.0,2019.0
1,5000,5000,5000.0,36,22.35%,191.86,D,D5,Director Of Sales,10+ years,MORTGAGE,72000.0,Source Verified,2018-12-01,Fully Paid,n,debt_consolidation,Debt consolidation,333xx,FL,20.12,0,2010-03-01,0,NaN,NaN,13,0,11720,47.1%,26,f,0.00,0.00,5615.977674,5615.98,5000.00,615.98,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,22.35,2018,12,3197,0,7.0,2019.0
2,10000,10000,10000.0,60,23.40%,284.21,E,E1,Unknown,< 1 year,RENT,55000.0,Source Verified,2018-12-01,Current,n,debt_consolidation,Debt consolidation,902xx,CA,13.51,0,2007-04-01,0,44.0,88.0,9,1,11859,53.9%,11,w,8694.39,8694.39,3675.230000,3675.23,1305.61,2369.62,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,23.40,2018,12,4262,1,1.0,2020.0
3,17100,17100,17100.0,36,18.94%,626.30,D,D2,Receptionist,10+ years,RENT,38000.0,Verified,2018-12-01,Current,n,debt_consolidation,Debt consolidation,150xx,PA,38.09,0,1998-03-01,1,47.0,NaN,14,0,15323,53%,21,w,12001.54,12001.54,8114.910000,8114.91,5098.46,3016.45,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,18.94,2018,12,7580,1,1.0,2020.0
4,4000,4000,4000.0,36,10.72%,130.43,B,B2,Extrusion Assistant,10+ years,MORTGAGE,56000.0,Verified,2018-12-01,Current,n,credit_card,Credit card refinancing,301xx,GA,31.03,0,2006-09-01,0,NaN,NaN,7,0,4518,28.6%,11,w,2700.74,2700.74,1682.500000,1682.50,1299.26,383.24,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,10.72,2018,12,4474,1,1.0,2020.0
5,10475,10475,10475.0,36,11.31%,344.48,B,B3,Teacher,6 years,RENT,66150.0,Source Verified,2018-12-01,Current,n,debt_consolidation,Debt consolidation,786xx,TX,7.40,0,1991-06-01,0,28.0,NaN,10,0,10499,40.2%,17,w,7093.15,7093.15,4468.370000,4468.37,3381.85,1086.52,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,11.31,2018,12,10045,1,1.0,2020.0
6,7500,7500,7500.0,36,11.31%,246.65,B,B3,Supervisor,7 years,MORTGAGE,40500.0,Ver

In [123]:
# There's one other column in the dataframe with percent signs. Remove them and convert to floats. You'll need to handle missing values.
# revol_util

loan_df['revol_util'].isnull().sum()


156

In [121]:
loan_df['revol_util'].

count     128256
unique      1074
top           0%
freq        1132
Name: revol_util, dtype: object

In [0]:
def obj_to_float(cell_inputs):
  if isinstance(cell_inputs, str):
    return float(cell_inputs.strip('%'))
  else:
    return "NaN"
  

In [0]:
loan_df['revol_util_float'] = loan_df['revol_util'].apply(obj_to_float)

In [135]:
loan_df['revol_util_int'].describe()


count     128412.0
unique      1075.0
top            0.0
freq        1132.0
Name: revol_util_int, dtype: float64

In [0]:
# Or this method
loan_df["revol_util_loans"] = loan_df['revol_util'].str.rstrip('%').astype('float')


In [186]:
loan_df['revol_util_loans']

0         43.2
1         47.1
2         53.9
3         53.0
4         28.6
          ... 
128407    43.6
128408    37.1
128409    27.3
128410    13.1
128411    30.8
Name: revol_util_loans, Length: 128412, dtype: float64

In [0]:
# Modify the emp_title column to replace titles with 'Other' if the title is not in the top 20.


In [217]:
loan_df['emp_title'].value_counts().head(20)

Unknown               20947
Teacher                2557
Manager                2395
Registered Nurse       1418
Driver                 1258
Supervisor             1160
Truck Driver            920
Rn                      834
Office Manager          805
Sales                   803
General Manager         791
Project Manager         720
Owner                   625
Director                523
Operations Manager      518
Sales Manager           500
Police Officer          440
Nurse                   425
Technician              420
Engineer                412
Name: emp_title, dtype: int64

In [281]:
top20_titles = df['emp_title'].value_counts().head(20).index

top20_titles

Index(['Teacher', 'Manager', 'Registered Nurse', 'Driver', 'RN', 'Supervisor',
       'Sales', 'Project Manager', 'General Manager', 'Office Manager',
       'Owner', 'Director', 'Operations Manager', 'Truck Driver', 'Nurse',
       'Engineer', 'Sales Manager', 'manager', 'Supervisor ',
       'Administrative Assistant'],
      dtype='object')

In [282]:
emp_list = loan_df['emp_title']
emp_list

0              Staff Physician
1            Director Of Sales
2                      Unknown
3                 Receptionist
4          Extrusion Assistant
                  ...         
128407          Tax Consultant
128408          Security Guard
128409            Payoff Clerk
128410                 Unknown
128411    Key Accounts Manager
Name: emp_title, Length: 128412, dtype: object

In [0]:
def findtitle(list):
  for row in list:
    if title in row[top20_titles]:
      return row
  else:
    return "Other"

In [292]:
emp_list.apply(findtitle).value_counts()

NameError: ignored

In [0]:
loan_df['top20'] = findemp

In [277]:
loan_df.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,...,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,int_rate_float,issue_year,issue_month,Credit History,loan_status_is_great,last_pymnt_d_month,last_pymnt_d_year,revol_util_int,revol_util_float,top20,revol_util_loans,loan_status_is_great1
0,35000,35000,35000.0,36,14.47%,1204.23,C,C2,Staff Physician,8 years,MORTGAGE,360000.0,Verified,2018-12-01,Fully Paid,n,credit_card,Credit card refinancing,336xx,FL,19.90,0,1995-04-01,1,NaN,NaN,24,0,57259,43.2%,51,w,0.00,0.00,38187.046837,38187.05,35000.00,3187.05,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,14.47,2018,12,8645,1,8.0,2019.0,43.2,43.2,Unknown,43.2,1
1,5000,5000,5000.0,36,22.35%,191.86,D,D5,Director Of Sales,10+ years,MORTGAGE,72000.0,Source Verified,2018-12-01,Fully Paid,n,debt_consolidation,Debt consolidation,333xx,FL,20.12,0,2010-03-01,0,NaN,NaN,13,0,11720,47.1%,26,f,0.00,0.00,5615.977674,5615.98,5000.00,615.98,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,22.35,2018,12,3197,1,7.0,2019.0,47.1,47.1,Unknown,47.1,1
2,10000,10000,10000.0,60,23.40%,284.21,E,E1,Unknown,< 1 year,RENT,55000.0,Source Verified,2018-12-01,Current,n,debt_consolidation,Debt consolidation,902xx,CA,13.51,0,2007-04-01,0,44.0,88.0,9,1,11859,53.9%,11,w,8694.39,8694.39,3675.230000,3675.23,1305.61,2369.62,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,23.40,2018,12,4262,1,1.0,2020.0,53.9,53.9,Unknown,53.9,1
3,17100,17100,17100.0,36,18.94%,626.30,D,D2,Receptionist,10+ years,RENT,38000.0,Verified,2018-12-01,Current,n,debt_consolidation,Debt consolidation,150xx,PA,38.09,0,1998-03-01,1,47.0,NaN,14,0,15323,53%,21,w,12001.54,12001.54,8114.910000,8114.91,5098.46,3016.45,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,18.94,2018,12,7580,1,1.0,2020.0,53,53,Unknown,53.0,1
4,4000,4000,4000.0,36,10.72%,130.43,B,B2,Extrusion Assistant,10+ years,MORTGAGE,56000.0,Verified,2018-12-01,Current,n,credit_card,Credit card refinancing,301xx,GA,31.03,0,2006-09-01,0,NaN,NaN,7,0,4518,28.6%,11,w,2700.74,2700.74,1682.500000,1682.50,1299.26,383.24,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,10.72,2018,12,4474,1,1.0,2020.0,28.6,28.6,Unknown,28.6,1


20947

# Stretch Goals

You can do more with the LendingClub or Instacart datasets.

LendingClub options:
- There's one other column in the dataframe with percent signs. Remove them and convert to floats. You'll need to handle missing values.
- Modify the `emp_title` column to replace titles with 'Other' if the title is not in the top 20. 
- Take initiatve and work on your own ideas!

Instacart options:
- Read [Instacart Market Basket Analysis, Winner's Interview: 2nd place, Kazuki Onodera](http://blog.kaggle.com/2017/09/21/instacart-market-basket-analysis-winners-interview-2nd-place-kazuki-onodera/), especially the **Feature Engineering** section. (Can you choose one feature from his bulleted lists, and try to engineer it with pandas code?)
- Read and replicate parts of [Simple Exploration Notebook - Instacart](https://www.kaggle.com/sudalairajkumar/simple-exploration-notebook-instacart). (It's the Python Notebook with the most upvotes for this Kaggle competition.)
- Take initiative and work on your own ideas!

You can uncomment and run the cells below to re-download and extract the Instacart data

In [0]:
# !wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

In [0]:
# !tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

In [0]:
# %cd instacart_2017_05_01